In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers, Model, mixed_precision
from tensorflow.keras.layers import Layer, Input, Conv2D, DepthwiseConv2D, ZeroPadding2D, BatchNormalization, ReLU, Add, Flatten
from tensorflow.keras import backend as K

import math
import os

physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

# mixed precision training is used
mixed_precision.set_global_policy('mixed_float16')

# 128 for 3060M, 512 for P100 
GLOBAL_BATCH_SIZE = 512
print('Global batch size: {}'.format(GLOBAL_BATCH_SIZE))

2024-03-22 12:49:08.648771: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 12:49:08.973647: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 12:49:08.973738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 12:49:09.036486: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 12:49:09.164253: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 12:49:09.165854: I tensorflow/core/platform/cpu_feature_guard.cc:1

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Global batch size: 512


In [2]:
def parse_function(example_proto, n_classes=85742):
    features = {'image_raw': tf.io.FixedLenFeature([], tf.string),
                'label': tf.io.FixedLenFeature([], tf.int64)}
    features = tf.io.parse_single_example(example_proto, features)
    img = tf.image.decode_jpeg(features['image_raw'])
    img = tf.reshape(img, shape=(112, 112, 3))

    # You can do more image distortion here for training data
    img = tf.cast(img, dtype=tf.float32)
    #img = tf.subtract(img, 127.5)
    img = tf.subtract(img, 128)
    img = tf.multiply(img,  0.0078125)
    img = tf.image.random_flip_left_right(img)
    label = tf.cast(features['label'], tf.int64)
    label = tf.one_hot(label, n_classes)
    return (img, label), label


file_dataset = tf.data.TFRecordDataset.list_files("faces-ms1m-refine-v2-112x112-tfrecord/faces_ms1m_refine_v2_112x112-*.tfrecord")
train_dataset = tf.data.TFRecordDataset(file_dataset,num_parallel_reads=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(parse_function,num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(GLOBAL_BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: faces-ms1m-refine-v2-112x112-tfrecord/faces_ms1m_refine_v2_112x112-*.tfrecord'

In [5]:
##### Model
class ArcFace(Layer):
    def __init__(self, n_classes=10, s=64.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)
        super(ArcFace, self).build(input_shape[0])

    def call(self, inputs):
        cos_m = math.cos(self.m)
        sin_m = math.sin(self.m)
        mm = sin_m * self.m
        threshold = math.cos(math.pi - self.m)

        embedding, labels = inputs

        embedding_norm = tf.norm(embedding, axis=1, keepdims=True)
        embedding = tf.divide(embedding, embedding_norm, name='norm_embedding')
        weights = self.W
        weights_norm = tf.norm(weights, axis=0, keepdims=True)
        weights = tf.divide(weights, weights_norm, name='norm_weights')
        # cos(theta+m)
        cos_t = tf.matmul(embedding, weights, name='cos_t')
        cos_t2 = tf.square(cos_t, name='cos_2')
        sin_t2 = tf.subtract(1., cos_t2, name='sin_2')
        sin_t = tf.sqrt(sin_t2, name='sin_t')
        cos_mt = self.s * tf.subtract(tf.multiply(cos_t, cos_m), tf.multiply(sin_t, sin_m), name='cos_mt')

        # this condition controls the theta+m should in range [0, pi]
        #      0<=theta+m<=pi
        #     -m<=theta<=pi-m
        cond_v = cos_t - threshold
        cond = tf.cast(tf.nn.relu(cond_v, name='if_else'), dtype=tf.bool)

        keep_val = self.s*(cos_t - mm)
        cos_mt_temp = tf.where(cond, cos_mt, keep_val)

        mask = labels
        inv_mask = tf.subtract(1., mask, name='inverse_mask')

        s_cos_t = tf.multiply(self.s, cos_t, name='scalar_cos_t')

        logit = tf.add(tf.multiply(s_cos_t, inv_mask), tf.multiply(cos_mt_temp, mask), name='arcface_loss_output')
        out = tf.nn.softmax(logit)
        
        return out


    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)
    
    def get_config(self):
        return {'s': self.s, 'm': self.m, 'n_classes': self.n_classes}

def correct_pad(inputs, kernel_size):
    img_dim = 2 if tf.keras.backend.image_data_format() == 'channels_first' else 1
    input_size = tf.keras.backend.int_shape(inputs)[img_dim:(img_dim + 2)]

    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)

    if input_size[0] is None:
        adjust = (1, 1)
    else:
        adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)

    correct = (kernel_size[0] // 2, kernel_size[1] // 2)

    return ((correct[0] - adjust[0], correct[0]),
            (correct[1] - adjust[1], correct[1]))


def inverted_res_block(inputs, expansion, stride, filters, block_id):
    channel_axis = -1
    in_channels = tf.keras.backend.int_shape(inputs)[channel_axis]
    pointwise_filters = filters
    x = inputs
    prefix = 'block_{}_'.format(block_id)

    # Expand
    x = Conv2D(
        expansion * in_channels,
        kernel_size=1,
        padding='same',
        use_bias=False,
        activation=None,
        name=prefix + 'expand')(x)
    x = BatchNormalization(
        axis=channel_axis,
        epsilon=1e-3,
        momentum=0.999,
        name=prefix + 'expand_BN')(x)

    # Depthwise
    if stride == 2:
        x = ZeroPadding2D(
            padding=correct_pad(x, 3),
            name=prefix + 'pad')(x)
    x = DepthwiseConv2D(
        kernel_size=3,
        strides=stride,
        activation=None,
        use_bias=False,
        padding='same' if stride == 1 else 'valid',
        name=prefix + 'depthwise')(x)
    x = BatchNormalization(
        axis=channel_axis,
        epsilon=1e-3,
        momentum=0.999,
        name=prefix + 'depthwise_BN')(x)

    x = ReLU(6., name=prefix + 'depthwise_relu')(x)

    # Project
    x = Conv2D(
        pointwise_filters,
        kernel_size=1,
        padding='same',
        use_bias=False,
        activation=None,
        name=prefix + 'project')(x)
    x = BatchNormalization(
        axis=channel_axis,
        epsilon=1e-3,
        momentum=0.999,
        name=prefix + 'project_BN')(x)

    if in_channels == pointwise_filters and stride == 1:
        return Add(name=prefix + 'add')([inputs, x])
    
    return x

def mobilefacenet_arcface(n_classes=85742):
    weight_decay=0.00005
    
    # input
    input = Input(shape=(112, 112, 3), name='input')
    y = Input(shape=(n_classes), name='label')

    # Conv2D
    x = Conv2D(64, (3, 3), padding='same', strides=(2, 2), kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(weight_decay), use_bias=False, name='sec1_conv2d')(input)
    x = BatchNormalization(name='sec1_bn')(x)
    x = ReLU(name='sec1_relu')(x)

    # DepthwiseConv2D
    x = DepthwiseConv2D((3,3), padding='same', strides=(1, 1), kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(weight_decay), use_bias=False, name='sec2_depthwiseconv2d')(x)
    x = BatchNormalization(name='sec2_bn')(x)
    x = ReLU(name='sec2_relu')(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(weight_decay), name='sec2_conv2d', use_bias=False)(x)
    x = BatchNormalization(name='sec2_bn2')(x)

    # InvertedResidualBlock
    x = inverted_res_block(x, 2, 2,  64, 0)
    x = inverted_res_block(x, 4, 2, 128, 1)
    x = inverted_res_block(x, 2, 1, 128, 2)
    x = inverted_res_block(x, 4, 2, 128, 3)
    x = inverted_res_block(x, 2, 1, 128, 4)

    # Conv2D 1x1
    x = Conv2D(512, (1, 1), padding='same', strides=(1, 1), kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(weight_decay), name='sec8_conv2d', use_bias=False)(x)
    x = BatchNormalization(name='sec8_bn')(x)
    x = ReLU(name='sec8_relu')(x)

    # linear GDConv 7x7
    x = DepthwiseConv2D((7,7), padding='valid', strides=(1, 1), kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(weight_decay), use_bias=False, name='sec9_depthwiseconv2d')(x)
    x = BatchNormalization(name='sec9_bn')(x)
    x = Conv2D(512, (1, 1), padding='valid', strides=(1, 1), kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(weight_decay), name='sec9_conv2d', use_bias=False)(x)
    x = BatchNormalization(name='sec9_bn2')(x)

    # linear Conv2D 1x1
    x = Conv2D(128, (1, 1), padding='same', strides=(1, 1), kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(weight_decay), use_bias=False, name='sec10_conv2d')(x)
    x = BatchNormalization(name='sec10_bn')(x)

    # faltten
    x = Flatten(dtype='float32')(x)

    # embedding
    x = tf.keras.layers.Lambda(lambda k: tf.keras.backend.l2_normalize(k, axis=-1), name="embedding")(x)

    # loss function
    output = ArcFace(n_classes=85742, dtype='float32')([x, y])

    return Model([input, y], output)

In [ ]:
model = mobilefacenet_arcface()
opt = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#model.summary()
checkpoint_filepath = './ckpt/epoch_{epoch:02d}'

def scheduler(epoch, lr):
    if epoch < 2:
        return 0.01
    elif epoch < 5:
        return 0.001
    else:
        return 0.0001

lr_cb = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False)

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="logs",
    write_graph=True,
    write_images=True,
    update_freq="epoch")

model.fit(train_dataset,
    epochs=7,
    callbacks=[checkpoint_cb, tensorboard_cb, lr_cb],
    verbose=1)


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/7


2024-03-17 18:09:56.613246: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 425852928 exceeds 10% of free system memory.


In [ ]:
# set the global policy to fp32 and rebuild a fp32 model, this is required for exporting to TFLite model

tf.keras.mixed_precision.set_global_policy("float32")
f32_model = mobilefacenet_arcface()
f32_model.set_weights(model.get_weights())

# extract the output model from the model used for training
outputModel = tf.keras.Model(f32_model.get_layer('input').input, f32_model.get_layer('embedding').output, trainable=False)

# save the trained model as TensorFlow SavedModel format
outputModel.save('/output_model')

# Convert the output model into a TensorflowLite model and save it
converter = tf.lite.TFLiteConverter.from_keras_model(outputModel)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('output_model/MobileFaceNet.tflite', 'wb') as f:
    f.write(tflite_model)